In [5]:
import numpy as np

# Read the input file into an array
lines = []
with open('input.txt') as f:
    lines = f.readlines()

#Defines position of beakers of samples. These are defined if you choose to print the standard layout, but can also be manually adjusted for other sample / beaker designs. Minor adjustments possible via the input file.
Beakerpos = np.array([[66, 155], [126, 155], [186, 155], [66, 221], [126, 221], [186, 221]])
Samplepos = np.array([[76, 207], [96, 207], [116, 207], [136, 207], [156, 207], [176, 207]])


#Reading the input array to get the initialization parameters
Steps = 15 - len(lines)
BeakerHeight = float(lines[8].split(" ")[-1])
LiquidHeight = float(lines[9].split(" ")[-1])
SampleHeight = float(lines[10].split(" ")[-1])
BedTemperature = float(lines[11].split(" ")[-1])


#Reading the input array to get all dipping steps into an array
Dippingparam = [0]*abs(Steps)
j = 0
while j < abs(Steps):
    Dippingparam1 = lines[15+j].split("\t")
    Dippingparam[j] = [float(i) for i in Dippingparam1]
    j+=1

#Correction for Beaker and Sample Positions if set in input file
j = 0
if str(lines[1].split(" ")[-1]) == "Y\n":
    Beakeradj = lines[2].split("(")[-1]
    Beakeradj = Beakeradj.split(")")[0]
    Beakeradj = Beakeradj.split(";")
    while j < len(Beakerpos):
        Beakerpos[j,0] = Beakerpos[j,0]+float(Beakeradj[j].split(",")[1])
        Beakerpos[j,1] = Beakerpos[j,1]+float(Beakeradj[j].split(",")[2])
        j += 1

j = 0
if str(lines[4].split(" ")[-1]) == "Y\n":
    Sampleadj = lines[5].split("(")[-1]
    Sampleadj = Sampleadj.split(")")[0]
    Sampleadj = Sampleadj.split(";")
    while j < len(Samplepos):
        Samplepos[j,0] = Samplepos[j,0]+float(Sampleadj[j].split(",")[1])
        Samplepos[j,1] = Samplepos[j,1]+float(Sampleadj[j].split(",")[2])
        j += 1

#Initialization to allow for all the functions to work correctly, such as: Homing position, setting max acceleration and speed, setting bed temperature and opening the gripper
Init =["M302 S0", "M203 X1000 Y1000 Z1000", "M201 X5000 Y5000 Z5000", "M350 Z16", "G28", "G1 F1000", "M140 S"+str(int(BedTemperature)),"G1 E3"]

#Creating the Dipping Movements by interpreting the Dippingparam-array line by line and appending the necessary g-code accordingly to the array Cont
j = 0
Cont = []
Dipping = []
while j < len(Dippingparam): 
    CurrentDip = Dippingparam[j]
    
#Special case to grab the first sample
    if j == 0:
        Grabbing = ["G1 Z"+str(BeakerHeight+SampleHeight+10), "G1 "+"X"+str(Samplepos[int(CurrentDip[0])][0])+" Y"+str(Samplepos[int(CurrentDip[0])][1]), "G1 Z"+str(SampleHeight), "G1 E0", "G1 Z"+str(BeakerHeight+SampleHeight+10)]
        Cont.append(Grabbing)

#Commands to store the previous sample and grab the next one
    else:
        PreviousDip = Dippingparam[j-1]
        #print(PreviousDip)
        if Dippingparam[j-1][0] != Dippingparam[j][0]:
            Storing = ["G1 Z"+str(BeakerHeight+SampleHeight+10), "G1 "+"X"+str(Samplepos[int(PreviousDip[0])][0])+" Y"+str(Samplepos[int(PreviousDip[0])][1]), "G1 Z"+str(SampleHeight+3), "G1 E3", "G1 Z"+str(BeakerHeight+SampleHeight+10)]
            Grabbing = ["G1 Z"+str(BeakerHeight+SampleHeight+10), "G1 "+"X"+str(Samplepos[int(CurrentDip[0])][0])+" Y"+str(Samplepos[int(CurrentDip[0])][1]), "G1 Z"+str(SampleHeight), "G1 E0", "G1 Z"+str(BeakerHeight+SampleHeight+10)]
            Cont.append(Storing)
            Cont.append(Grabbing)
            
#Moving the sample to the defined beaker
    ToBeaker = ["G1 Z"+str(BeakerHeight+SampleHeight+10), "G1 "+"X"+str(Beakerpos[int(CurrentDip[1])][0])+" Y"+str(Beakerpos[int(CurrentDip[1])][1])]
#Dipping the sample 
    Dipping = ["G1 F"+str(int(CurrentDip[3])), "M204 P"+str(int(CurrentDip[4])),  "M204 T"+str(int(CurrentDip[4])), "G1 Z" + str(LiquidHeight+SampleHeight-CurrentDip[2]), "G4 P"+str(CurrentDip[5]*1000), "G1 F"+str(int(CurrentDip[7])), "M204 P"+str(int(CurrentDip[8])),  "M204 T"+str(int(CurrentDip[8])), "G1 Z"+str(LiquidHeight+SampleHeight+CurrentDip[10]),"G1 F1000", "G4 P"+str(CurrentDip[9]*1000)]
    Cont.append(ToBeaker)
    Cont.append(Dipping)
    j += 1
    
#Special case to store the last sample
    if j == len(Dippingparam):
        Storing = ["G1 Z"+str(BeakerHeight+SampleHeight+10), "G1 "+"X"+str(Samplepos[int(CurrentDip[0])][0])+" Y"+str(Samplepos[int(CurrentDip[0])][1]), "G1 Z"+str(SampleHeight+3), "G1 E2", "G1 Z"+str(BeakerHeight+SampleHeight+10), "G1 X0 Y0", "G1 E0"]
        Cont.append(Storing)

#Convert the array Cont to a writable list for exporting to a txt. file
Procedure = [item for sublist in Cont for item in sublist]

#Write to a output file
with open('out.gcode', 'w') as f:
    f.write('\n'.join(Init))
    f.write('\n')
    f.write('\n'.join(Procedure))